# Milestone 1: Political things
Author: Leonardo Matone 
Webpage: 

Project Description:

Project Goals:

Work plan:
- what technologies you are using
- when / how often you are meeting to work on this project

Data/Resources:
- 1 Question
- ETL
- 2 Interesting stats
- 2 Graphs